# Transportation optimization

In [53]:
import sys
sys.path.append('D:/source/repos')
from utilities.std_imports import *

### Create Transport Plan

### Testing

In [54]:
data = pd.read_csv('D:/data/csv/supply_chain/delivery_records.csv')
data

,Date,Truck_ID,Store_ID,FTL,Order,BOX,SKU,Loading_T
0,9/1/2016,Truck_ID1,Store_ID1,3.5,16,311,83,2.404
1,9/1/2016,Truck_ID1,Store_ID2,3.5,18,178,83,1.668
2,9/1/2016,Truck_ID2,Store_ID3,3.5,10,74,54,0.810
3,9/1/2016,Truck_ID2,Store_ID4,3.5,19,216,88,2.413
4,9/1/2016,Truck_ID3,Store_ID5,3.5,10,117,54,1.119
5,9/1/2016,Truck_ID3,Store_ID6,3.5,15,294,92,2.962
6,9/1/2016,Truck_ID4,Store_ID7,3.5,5,42,19,0.421
7,9/1/2016,Truck_ID4,Store_ID8,3.5,12,125,88,1.138
8,9/1/2016,Truck_ID5,Store_ID9,5.0,18,201,95,2.190
9,10/1/2016,Truck_ID1,Store_ID1,3.5,16,311,83,2.404


In [56]:
capacity_dict = {'Truck_ID1': 3.5, 'Truck_ID2': 3.5, 'Truck_ID3': 3.5, 'Truck_ID4': 3.5, 'Truck_ID5': 5.0}

df_plan = pd.DataFrame(data.groupby(['Date', 'Truck_ID'])['Store_ID'].apply(list))
df_plan.columns = ['Stores']
df_plan['Box_Qty'] = data.groupby(['Date', 'Truck_ID'])['BOX'].apply(list)

# Mean of FTL
df_plan['FTL']  = data.groupby(['Date', 'Truck_ID'])['FTL'].mean()
#df_plan['Capacity(T)'] = df_plan['FTL'].map(capacity_dict)
df_plan['List_Loading'] = data.groupby(['Date', 'Truck_ID'])['Loading_T'].apply(list)
df_plan['Count'] = df_plan['List_Loading'].apply(lambda t: len(t))
df_plan['Total_tons(T)'] = data.groupby(['Date', 'Truck_ID'])['Loading_T'].sum()
    
# Distribute: one shipment per col
# Stores
d = df_plan['Stores'].apply(pd.Series)
for col in d:
    df_plan["Store%d" % (col+1)] = d[col]
 
# Boxes number    
d = df_plan['Box_Qty'].apply(pd.Series)
for col in d:
    df_plan["Box%d" % (col+1)] = d[col]

# Shipments Tonnage
d = df_plan['List_Loading'].apply(pd.Series)
for col in d:
    df_plan["Tons%d" % (col+1)] = d[col]

df_plan.fillna(0, inplace = True)
if 1 == 0:
    df_plan.drop(['Stores'], axis = 1, inplace = True)
    df_plan.drop(['Box_Qty'], axis = 1, inplace = True)
    df_plan.drop(['List_Loading'], axis = 1, inplace = True)
df_plan

Stores     Box_Qty  FTL  \
Date      Truck_ID                                             
10/1/2016 Truck_ID1  [Store_ID1, Store_ID2]  [311, 178]  3.5   
          Truck_ID2  [Store_ID3, Store_ID4]   [74, 216]  3.5   
          Truck_ID3  [Store_ID5, Store_ID6]  [117, 294]  3.5   
          Truck_ID4  [Store_ID7, Store_ID8]   [42, 125]  3.5   
          Truck_ID5             [Store_ID9]       [201]  5.0   
11/1/2016 Truck_ID1  [Store_ID1, Store_ID2]  [311, 178]  3.5   
          Truck_ID2  [Store_ID3, Store_ID4]   [74, 216]  3.5   
          Truck_ID3  [Store_ID5, Store_ID6]  [117, 294]  3.5   
          Truck_ID4  [Store_ID7, Store_ID8]   [42, 125]  3.5   
          Truck_ID5             [Store_ID9]       [201]  5.0   
9/1/2016  Truck_ID1  [Store_ID1, Store_ID2]  [311, 178]  3.5   
          Truck_ID2  [Store_ID3, Store_ID4]   [74, 216]  3.5   
          Truck_ID3  [Store_ID5, Store_ID6]  [117, 294]  3.5   
          Truck_ID4  [Store_ID7, Store_ID8]   [42, 125]  3.5   
          Truck_ID5             [Store_ID9]       [201]  5.0   

                                     List_Loading  Count  Total_tons(T)  \
Date      Truck_ID                                                        
10/1/2016 Truck_ID1   [2.404, 1.6680000000000001]      2          4.072   
          Truck_ID2    [0.81, 2.4130000000000003]      2          3.223   
          Truck_ID3   [1.119, 2.9619999999999997]      2          4.081   
          Truck_ID4  [0.42100000000000004, 1.138]      2          1.559   
          Truck_ID5                        [2.19]      1          2.190   
11/1/2016 Truck_ID1   [2.404, 1.6680000000000001]      2          4.072   
          Truck_ID2    [0.81, 2.4130000000000003]      2          3.223   
          Truck_ID3   [1.119, 2.9619999999999997]      2          4.081   
          Truck_ID4  [0.42100000000000004, 1.138]      2          1.559   
          Truck_ID5                        [2.19]      1          2.190   
9/1/2016  Truck_ID1   [2.404, 1.6680000000000001]      2          4.072   
          Truck_ID2    [0.81, 2.4130000000000003]      2          3.223   
          Truck_ID3   [1.119, 2.9619999999999997]      2          4.081   
          Truck_ID4  [0.42100000000000004, 1.138]      2          1.559   
          Truck_ID5                        [2.19]      1          2.190   

                        Store1     Store2   Box1   Box2  Tons1  Tons2  
Date      Truck_ID                                                     
10/1/2016 Truck_ID1  Store_ID1  Store_ID2  311.0  178.0  2.404  1.668  
          Truck_ID2  Store_ID3  Store_ID4   74.0  216.0  0.810  2.413  
          Truck_ID3  Store_ID5  Store_ID6  117.0  294.0  1.119  2.962  
          Truck_ID4  Store_ID7  Store_ID8   42.0  125.0  0.421  1.138  
          Truck_ID5  Store_ID9          0  201.0    0.0  2.190  0.000  
11/1/2016 Truck_ID1  Store_ID1  Store_ID2  311.0  178.0  2.404  1.668  
          Truck_ID2  Store_ID3  Store_ID4   74.0  216.0  0.810  2.413  
          Truck_ID3  Store_ID5  Store_ID6  117.0  294.0  1.119  2.962  
          Truck_ID4  Store_ID7  Store_ID8   42.0  125.0  0.421  1.138  
          Truck_ID5  Store_ID9          0  201.0    0.0  2.190  0.000  
9/1/2016  Truck_ID1  Store_ID1  Store_ID2  311.0  178.0  2.404  1.668  
          Truck_ID2  Store_ID3  Store_ID4   74.0  216.0  0.810  2.413  
          Truck_ID3  Store_ID5  Store_ID6  117.0  294.0  1.119  2.962  
          Truck_ID4  Store_ID7  Store_ID8   42.0  125.0  0.421  1.138  
          Truck_ID5  Store_ID9          0  201.0    0.0  2.190  0.000

In [19]:
tpm = pd.read_csv('D:/data/csv/supply_chain/trasportation_plan_model.csv')
tpm

,Date,TruckID,List_Code,Capacity(T),List_Loading,Count,Total_tons(T),Store1,Store2,Store3,...,Box1,Box2,Box3,Box4,Tons1,Tons2,Tons3,Tons4,Occupation(%),Available(T)
0,9/1/2016,Truck_ID1,['Store_ID6'],3.5,[2.91],1,2.91,ID6,0,0,...,243,0,0,0,2.91,0.00,0.00,0,83.14,0.59
1,9/1/2016,Truck_ID2,"['Store_ID34', 'Store_ID22', 'Store_ID9']",3.5,"[0.3, 1.37, 0.47]",3,2.14,ID34,ID22,ID9,...,31,116,44,0,0.30,1.37,0.47,0,61.14,1.36
2,9/1/2016,Truck_ID3,['Store_ID18'],3.5,[1.5],1,1.50,ID18,0,0,...,174,0,0,0,1.50,0.00,0.00,0,42.86,2.00
3,9/1/2016,Truck_ID4,['Store_ID37'],3.5,[2.3],1,2.30,ID37,0,0,...,179,0,0,0,2.30,0.00,0.00,0,65.71,1.20
4,9/1/2016,Truck_ID5,"['Store_ID34', 'Store_ID48']",3.5,"[2.14, 0.51]",2,2.65,ID34,ID48,0,...,168,46,0,0,2.14,0.51,0.00,0,75.71,0.85


In [3]:
## Pricing Functions
def f_maxcity(list_cities, list_price):
    return list_cities[list_price.index(max(list_price))] # Index of Maximum Price
def inner_stops(list_cities, max_city):
    return list_cities.count(max_city) - 1
def outer_stops(list_cities, max_city):
    return len(list_cities) - (list_cities.count(max_city))
def total_price(max_price, inner_stops, outer_stops, inner_price, outer_price):
    return max_price + inner_stops * inner_price + outer_stops * outer_price

# Calculate Price
def plan_price(df_strinfo, df_plan, inner_price, outer_price):

    # Dictionnary Ville
    dict_ville = dict(zip(df_strinfo.Code.values, df_strinfo.City.values))

    # Price per Truck Size： 3.5T, 5T, 8T
    dict_35, dict_5, dict_8 = [dict(zip(df_strinfo.City.values, df_strinfo[col].values)) for col in ['3.5T', '5T', '8T']]

    # Mapping Cities
    f_ville = lambda t: [dict_ville[i] for i in t] # literal_eval(t)

    # Mapping Price
    f_35 = lambda t: [dict_35[i] for i in t]
    f_5 = lambda t: [dict_5[i] for i in t]
    f_8 = lambda t: [dict_8[i] for i in t]

    # Mapping Price
    df_plan['List_City'] = df_plan['List_Code'].map(f_ville)
    df_plan['List_Price35'] = df_plan['List_City'].map(f_35)
    df_plan['List_Price5'] = df_plan['List_City'].map(f_5)
    df_plan['List_Price8'] = df_plan['List_City'].map(f_8)

    # Maximum Price City 
    f_maxprice = lambda t: max(t) # Maximum Price

    # Mapping First City
    df_plan['Max_Price35'] = df_plan['List_Price35'].map(f_maxprice)
    df_plan['Max_Price5'] = df_plan['List_Price5'].map(f_maxprice)
    df_plan['Max_Price8'] = df_plan['List_Price8'].map(f_maxprice)
    df_plan['Max_City'] = df_plan.apply(lambda x: f_maxcity(x.List_City, x.List_Price35), axis = 1)

    # Inner City Stop 
    df_plan['Inner_Stops'] = df_plan.apply(lambda x: inner_stops(x.List_City, x.Max_City), axis = 1)
    df_plan['Outer_Stops'] = df_plan.apply(lambda x: outer_stops(x.List_City, x.Max_City), axis = 1)

    # Total Price
    df_plan['Price35'] = df_plan.apply(lambda x: total_price(x.Max_Price35, x.Inner_Stops, x.Outer_Stops, 
        inner_price, outer_price), axis = 1)
    df_plan['Price5'] = df_plan.apply(lambda x: total_price(x.Max_Price5, x.Inner_Stops, x.Outer_Stops, 
        inner_price, outer_price), axis = 1)
    df_plan['Price8'] = df_plan.apply(lambda x: total_price(x.Max_Price8, x.Inner_Stops, x.Outer_Stops, 
        inner_price, outer_price), axis = 1)

    return df_plan